In [1]:
import json
import torch
import random
import datasets
from datasets import load_dataset, Dataset, load_from_disk

data = json.load(open("/home/mcwave/code/test/eqs_flan_template.json"))
questions = []
answers = []
templates = []
it = 0
for eq in data:
    questions.append(eq['question'])
    expr = eq["answer"]
    new_expr = expr
    found = False
    for i in range(len(expr) - 1):
        if expr[i:i + 2] == "e-":
            found = True
            new_expr = new_expr.replace(expr[i - 5:i + 3], "0")
    answers.append(new_expr)
    templates.append(eq["template"])
    it += 1
    if it == 1000:
        break
random.shuffle(answers)
ds = Dataset.from_dict({'question': questions, 'answer': answers})
train_ds = ds.train_test_split(test_size=0.04)

train_ds

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 960
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 40
    })
})

In [2]:
import numpy
import time
import math
import sympy as sp
from math import sin, cos, tan, sinh, cosh, tanh, asin, acos, atan, sqrt, log
import time
import numpy as np
import psutil

SEQ_LEN = 128
RANGE_START = 0.001
RANGE_END = 4
EMBED_SIZE = 768
LOWER_BOUND = -10
UPPER_BOUND = 10

# Input:
#    formula: A formula containing "x", which will be replaced to numbers between range_start and range_end
# Output:
#    a sequence of embeddings, each has embed_size dimensions, and each dimension is between LOWER_BOUND and UPPER_BOUND
total_points = SEQ_LEN * EMBED_SIZE
step = (RANGE_END - RANGE_START) / (total_points - 1)
x = sp.Symbol("x")

def generate_seq_embed(expr):
    f = sp.lambdify(x, expr, "numpy")
    nums = np.arange(RANGE_START, RANGE_END + step, step)
    # print(f(nums))
    # res = list(np.concatenate((f(nums), np.zeros(129 * 768)), axis = 0))
    res = f(nums)
    if np.iscomplexobj(res):
        return 1/0
    res = np.concatenate((res, np.zeros(129 * 768)), axis = 0)
    return res
    # x = RANGE_START
    # seq = [0] * (SEQ_LEN + 129) * 768
    # seq[0] = expr(RANGE_START)
    # for i in range(1, total_points):
    #     try:
    #         y = expr(x)
    #         seq[i] = max(LOWER_BOUND, min(UPPER_BOUND, y))
    #         # seq[i] = next(gen)
    #     except:
    #         seq[i] = seq[i - 1]
    #     x += step
    # return seq

# exprs = []
# embeds = []
# i = 0
# start_time = time.time()
# ds = Dataset.from_dict({'question': [], "answer": [], "inputs_embeds": []})
# empty = Dataset.from_dict({'question': [], "answer": [], "inputs_embeds": []})
# temp = empty
# for expr in answers:
    
#     # print(expr)
#     if psutil.virtual_memory().percent > 75.0:
#         break
#     try:
#         values = generate_seq_embed(expr)
#         temp = temp.add_item({"question": "[BOS]", "answer": templates[i], "inputs_embeds": values})
#     except:
#         pass
#     i += 1
#     if i % 100 == 0:
#         ds = datasets.concatenate_datasets([ds, temp], axis=0)
#         temp = empty
#         print(ds)
#     if i % 500 == 0:
#         print(i, time.time() - start_time)
#         start_time = time.time()

torch.Tensor(generate_seq_embed("sin(x)")).shape
# # i = 0
# # def add_embeds(expr):
# #     print(1234)
# #     expr["inputs_embeds"] = embeds[i]
# #     i += 1
# #     return expr
# # print("map")
# ds = ds.add_column("inputs_embeds", embeds)

# train_ds = ds.train_test_split(test_size=0.04)
# train_ds["train"].save_to_disk("train_embeds_templates:1000.hf")
# print("Saved train")
# train_ds["test"].save_to_disk("test_embeds_templates:1000.hf")
# print("Saved test")
# train_ds

torch.Size([197376])

In [3]:
# train_ds["train"] = datasets.load_from_disk("train_embeds_templates:100000.hf")
# train_ds["test"] = datasets.load_from_disk("test_embeds_templates:100000.hf")

In [4]:
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-base"
# MODEL_NAME = "results/2/checkpoint-1000"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
if tokenizer.bos_token is None:
    tokenizer.add_special_tokens({'bos_token': '[BOS]'})

CONTEXT_LENGTH = 128

# We prefix our tasks with "answer the question"
prefix = ""

# Define the preprocessing function

def preprocess_function(examples):
    """Add prefix to the sentences, tokenize the text, and set the labels"""
    # The "inputs" are the tokenized answer:
    inputs = [prefix + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=CONTEXT_LENGTH, truncation=True)
  
    # The "labels" are the tokenized outputs:
    labels = tokenizer(text_target=examples["answer"], 
                        max_length=CONTEXT_LENGTH,         
                        truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def add_padding(val):
    val["labels"] = val["labels"][1:]
    start_pad_tokens = [0] * 128
    end_pad_tokens_ids = [0] * (129 - len(val["input_ids"]))
    end_pad_tokens_labels = [0] * (256 - len(val["labels"]))
    val["input_ids"] = start_pad_tokens + val["input_ids"] + end_pad_tokens_ids
    val["labels"] = [32100] + val["labels"] + end_pad_tokens_labels
    val["attention_mask"] = [1] * 128 + val["attention_mask"] + end_pad_tokens_ids
    val["inputs_embeds"] = list(val["inputs_embeds"])
    return val

tokenized_dataset = train_ds#.map(preprocess_function, batched=True, remove_columns = ["question", "answer"], num_proc = 4).map(add_padding)
# tokenized_dataset["train"].save_to_disk("train_dataset_templates:100000.hf")
# print("Saved train")
# tokenized_dataset["test"].save_to_disk("test_dataset_templates:100000.hf")
# print("Saved test")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# t = tokenized_dataset["test"]
# mask = t["attention_mask"]
# ids = t["input_ids"]
# labels = t["labels"]
# for i in range(len(t)):
#     if len(mask[i]) != 257:
#         print("mask", i)
#     if len(ids[i]) != 257:
#         print("ids", i)
#     if len(labels[i]) != 257:
#         print("labels", i)
#     if i % 1000 == 0:
#         print(i)

In [6]:
tokenized_dataset["train"] = datasets.load_from_disk("train_dataset_templates:100000.hf")
tokenized_dataset["test"] = datasets.load_from_disk("test_dataset_templates:100000.hf")

In [7]:
torch.reshape(torch.Tensor(tokenized_dataset["train"][102]["inputs_embeds"]), (257, 768))

tensor([[-18.9315, -18.8119, -18.6968,  ...,  -8.5230,  -8.5192,  -8.5154],
        [ -8.5116,  -8.5079,  -8.5041,  ...,  -6.4849,  -6.4829,  -6.4810],
        [ -6.4791,  -6.4772,  -6.4753,  ...,  -5.2824,  -5.2811,  -5.2798],
        ...,
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000]])

In [8]:
import nltk
import evaluate
import numpy as np

nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 16

PER_DEVICE_EVAL_BATCH = 16
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 100
NUM_EPOCHS = 10
SAVE_STEPS=100

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="results/2/",
   evaluation_strategy="steps",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   save_steps=SAVE_STEPS,
   eval_steps=SAVE_STEPS,
   logging_steps=SAVE_STEPS,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


dec tensor([[    0, 32100,    18,  ...,     0,     0,     0],
        [    0, 32100,   152,  ...,     0,     0,     0],
        [    0, 32100,     9,  ...,     0,     0,     0],
        ...,
        [    0, 32100,   599,  ...,     0,     0,     0],
        [    0, 32100,     9,  ...,     0,     0,     0],
        [    0, 32100,     9,  ...,     0,     0,     0]], device='cuda:0')
None
None
seq tensor([[[-0.1322, -0.1135, -0.1459,  ...,  0.0469,  0.0588,  0.1472],
         [-0.0760, -0.1236, -0.0156,  ...,  0.0734,  0.0972,  0.2680],
         [ 0.0796, -0.0000, -0.0631,  ...,  0.0492,  0.1160,  0.1494],
         ...,
         [-0.0401,  0.0226,  0.0045,  ...,  0.2173, -0.0160,  0.0018],
         [-0.0955,  0.0078, -0.0160,  ...,  0.2408,  0.0333,  0.0198],
         [-0.1105, -0.0449, -0.0209,  ...,  0.0948,  0.0643,  0.1183]],

        [[-0.2503, -0.1373, -0.3209,  ...,  0.0000, -0.0678,  0.0000],
         [-0.2102,  0.0425, -0.3731,  ..., -0.0010, -0.0160,  0.0718],
         [-0.1555, 

Step,Training Loss,Validation Loss


dec tensor([[    0, 32100,   599,  ...,     0,     0,     0],
        [    0, 32100,    18,  ...,     0,     0,     0],
        [    0, 32100,     7,  ...,     0,     0,     0],
        ...,
        [    0, 32100,     7,  ...,     0,     0,     0],
        [    0, 32100,     9,  ...,     0,     0,     0],
        [    0, 32100,    17,  ...,     0,     0,     0]], device='cuda:0')
None
None
seq tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

       

dec tensor([[    0, 32100,   599,  ...,     0,     0,     0],
        [    0, 32100,     9,  ...,     0,     0,     0],
        [    0, 32100,     9,  ...,     0,     0,     0],
        ...,
        [    0, 32100,    17,  ...,     0,     0,     0],
        [    0, 32100,   152,  ...,     0,     0,     0],
        [    0, 32100,    52,  ...,     0,     0,     0]], device='cuda:0')
None
None
seq tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

       


KeyboardInterrupt



In [ ]:
# Encode some input text 
generate_seq_embed("-sin(1.94*x - 1.94)")
inputs_embeds = torch.reshape(torch.Tensor(generate_seq_embed("-sin(1.94*x - 1.94)")), [1, 257, 768]).to("cuda")
print(inputs_embeds)
prompt = "[BOS]"
input_ids = tokenizer.encode(prompt, return_tensors='pt').to("cuda")
print(input_ids)
# Generate text
output = model.generate(input_ids = input_ids, labels = input_ids, inputs_embeds = inputs_embeds, max_new_tokens = 50, num_return_sequences=1, temperature=0.7)

# Decode and print the output
generated_text = tokenizer.decode(output[0])
print(output, generated_text)

In [ ]:
x = torch.Tensor([1,2,3])
x.device

In [ ]:
# print(torch.reshape(torch.Tensor(generate_seq_embed("0.123*x**2+0.016*x-0.707")), [1, 257, 768])[:, :128].to("cuda"))
# print(torch.reshape(torch.Tensor(generate_seq_embed(eval("lambda x: 1.361*math.cos(2.0*x) - 0.167"))), [1, 257, 768])[:, :128].to("cuda"))